# Iterative algorithms

Test piecewise algorithms, constructing different trajectory models over time. This is still work in progress, but the functioning of two example algorithms is proven on simulated data in this notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from iterative_algorithms import *


%matplotlib inline
#%matplotlib notebook
%reload_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = 10, 5
np.set_printoptions(precision=5, suppress=True)

# Toy example

In [ ]:
from trajectory import Trajectory

np.random.seed(1)
t1 = Trajectory(dim=2, n_complexity=2, model='polynomial', 
                coeffs=np.array([[0., 0.], [0., 1.]]))
t2 = Trajectory(dim=2, n_complexity=2, model='polynomial', 
                coeffs=np.array([[-3., 3.], [1., 0.]]))
times1 = np.linspace(0, 1, 10)
times2 = np.linspace(1, 2, 20)
anchors = 4 * np.random.rand(2, 5)
fig, ax = plt.subplots()
ax.scatter(*anchors, color='red')
t1.plot(times=times1, ax=ax, color='black')
t2.plot(times=times2, ax=ax, color='black')
ax.axis('equal')

from measurements import get_measurements

b1, D1 = get_measurements(t1, anchors, seed=1, times=times1)
b2, D2 = get_measurements(t2, anchors, seed=1, times=times2)
F = np.hstack((b1, b2))
D = np.vstack((D1, D2))
times = np.r_[times1, times2]
print(D.shape, F.shape)

#plt.matshow(D)

In [ ]:
## Convert toy example to something more realistic. 
df = pd.DataFrame(columns=['timestamp', 'anchor_id', 'distance'])
i = 0

anchor_ids = range(anchors.shape[1])
print(anchor_ids)

noise = 1e-3
np.random.seed(1)

for d_mn_row, time in zip(D, times):
    anchor_indexes = np.where(d_mn_row>0)[0]
    for anchor_idx in anchor_indexes:
        distance=d_mn_row[anchor_idx]
        if noise > 0:
            distance += np.random.normal(scale=noise)
        df.loc[i, :] = dict(
            timestamp=time,
            anchor_id=anchor_ids[anchor_idx],
            distance=distance
        )
        i += 1
df.head()

### 1. Averaging algorithm

Estimate the trajectory recursively over a fixed  time window.

In [ ]:
C_list, t_list = averaging_algorithm(D, anchors, F, times, t_window=1.0)

np.testing.assert_allclose(C_list[0], t1.coeffs)
np.testing.assert_allclose(C_list[-2], t2.coeffs)
np.testing.assert_allclose(C_list[-1], t2.coeffs)

fig, ax = plt.subplots()
for C, t in zip(C_list, t_list):
    trajk = t1.copy()
    trajk.set_coeffs(coeffs=C)
    trajk.plot(ax=ax, times=t)
fig.set_size_inches(5, 3)
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.tight_layout()
fig.savefig('results/moving_window.png')

### 2. Build-up algorithm

Refine the current trajectory as long as the new measurements "fit the model" well enough. 

Create a completely new trajectory model with measurements if they stop fitting. 

In [ ]:
C_list, t_list = build_up_algorithm(D, anchors, F, times, eps=1e-3)

np.testing.assert_allclose(C_list[0], t1.coeffs)
np.testing.assert_allclose(C_list[1], t2.coeffs)
    
fig, ax = plt.subplots()
for C, t in zip(C_list, t_list):
    trajk = t1.copy()
    trajk.set_coeffs(coeffs=C)
    trajk.plot(ax=ax, times=t)
fig.set_size_inches(5, 3)
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.tight_layout()
fig.savefig('results/iterative.png')

# Real example

In [ ]:
from public_data_utils import *
#filename = 'datasets/uah1.mat' # fingers
#filename = 'datasets/Plaza1.mat'; # zig zag. 
filename = 'datasets/Plaza2.mat' # triangle

full_df, anchors_df, traj = read_dataset(filename)
xlim, ylim = get_plotting_params(filename)

In [ ]:
from evaluate_dataset import compute_distance_matrix, compute_anchors

chosen_df = full_df
#chosen_df = filtered_df
chosen_distance = 'distance'
#chosen_distance = 'distance_gt'
anchor_names = None

## Construct anchors. 
anchors = compute_anchors(anchors_df, anchor_names)

## Construct times.
times = chosen_df[chosen_df.system_id == range_system_id].timestamp.unique()

## Construct D.
D, times = compute_distance_matrix(chosen_df, anchors_df, anchor_names, times, chosen_distance)
if np.sum(D > 0) > D.shape[0]:
    print('Warning: multiple measurements for times:{}/{}!'.format(
          np.sum(np.sum(D > 0, axis=1)>1), D.shape[0]))

## Construct ground truth.
points_gt = get_ground_truth(chosen_df, times)

In [ ]:
period_it = 10
dataname = filename.split('/')[-1].split('.')[0]
if dataname == 'uah1':
    # for iterative.
    n_complexity_it = 2
    model_it = 'polynomial'
    t_window_it = 80
    eps = 2.0
elif dataname == 'Plaza1':
    # for iterative.
    n_complexity_it = 3
    model_it = 'full_bandlimited'
    period_it = 40
    t_window_it = 20
    eps = 0.5
elif dataname == 'Plaza2':
    # for iterative.
    n_complexity_it = 3
    model_it = 'bandlimited'
    period_it = 40
    t_window_it = 40
    eps = 0.2

In [ ]:
traj_it = traj.copy()
traj_it.set_n_complexity(n_complexity_it)
traj_it.model = model_it
traj_it.period = period_it
basis = traj_it.get_basis(times=times)
print('Using trajectory model: \n model={}, K={}, period={}'.format(traj_it.model, traj_it.n_complexity, traj_it.period))

### 1. Averaging algorithm

In [ ]:
print('averaging with time window', t_window_it)
C_list, t_list = averaging_algorithm(D, anchors[:2, :], basis, times, t_window=t_window_it)
ax1 = plot_individual(C_list, t_list, traj_it)
ax1.plot(points_gt.px, points_gt.py, color='black')
result_df = get_smooth_points(C_list, t_list, traj_it)
ax2 = plot_smooth(result_df)
ax2.plot(points_gt.px, points_gt.py, color='black')
[[ax.set_xlim(*xlim), ax.set_ylim(*ylim)] for ax in [ax1, ax2]]

### 2. Build-up algorithm

In [ ]:

C_list, t_list = build_up_algorithm(D, anchors[:2, :], basis, times, eps=eps, verbose=False)
ax1 = plot_individual(C_list, t_list, traj_it.copy())
ax1.plot(points_gt.px, points_gt.py, color='black')

result_df = get_smooth_points(C_list, t_list, traj_it)
ax2 = plot_smooth(result_df)
ax2.plot(points_gt.px, points_gt.py, color='black')
[[ax.set_xlim(*xlim), ax.set_ylim(*ylim)] for ax in [ax1, ax2]]